In [1]:
#!/usr/bin/env python
# -*- coding: latin-1 -*-
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import logging
import difflib
import Levenshtein as lev
from tqdm.notebook import tqdm
import multiprocessing
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import spatial
import re
import movies_df_helper_functions as helper_fn
import evaluation_helper_functions as eval_fn
import random
import imp
import itertools
from sklearn.manifold import TSNE
from sklearn.cluster import AffinityPropagation
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import html
import csv
import re
#from sklearn.manifold import TSNE

from tabulate import tabulate
import decimal
from collections import Counter

imp.reload(eval_fn)
imp.reload(helper_fn)
%matplotlib inline

In [2]:
#load train data
#train_data_df = pd.read_csv('models/baseline_exp/train_data.csv')
train_data_df = pd.read_csv('train_data/train_data.csv')
#load genres test data
genres_sample_df = pd.read_csv('test_data/genres_sample_movies_id_list.csv')
#load all genres lists df
genres_df = pd.read_csv('test_data/movies_by_genre.csv')
#load all sequel and mixed data for plotting
sequel_sampled_data_df = pd.read_csv('test_data/sequel_and_sampled_titles_df.csv')

In [3]:
#load model
model = Doc2Vec.load('baseline-exp/model')

FileNotFoundError: [Errno 2] No such file or directory: 'baseline-exp/model'

In [5]:
def assign_genre(genre_df, movie_id):
    romantic_comedy= genre_df.romantic_comedy.values.tolist()
    drama = genre_df.drama.values.tolist()
    adventure = genre_df.adventure.values.tolist()
    crime = genre_df.crime.values.tolist()
    action = genre_df.action.values.tolist()
    
    if movie_id in romantic_comedy:
        return 'Romantic_Comedy'
    else:
        if movie_id in drama:
            return 'Drama'
        else: 
            if movie_id in adventure:
                return 'Adventure'
            else: 
                if movie_id in crime:
                    return 'Crime'
                else: 
                    if movie_id in action: return 'Action'
                    else: 
                        return 'Unkown'

In [ ]:
train_data_df

In [6]:
#Clean train_data_df
#remove the square brackets saved when generating the test data.It i snot a bug. Data can't be saved without as they act as a column separator in the csv file
train_data_df['actor_name'] = train_data_df.actor_name.apply(lambda actor: actor.replace('[','').replace(']','').replace("'", "") )
train_data_df['director_name'] = train_data_df.director_name.apply(lambda director: director.replace('[','').replace(']','').replace("'", "") )
train_data_df['writer_name'] = train_data_df.writer_name.apply(lambda writer: writer.replace('[','').replace(']','').replace("'", "") )
train_data_df['country'] = train_data_df.country.apply(lambda country: country.replace('[','').replace(']','').replace("'", "") )
train_data_df['genre'] = train_data_df.genre.apply(lambda genre: genre.replace('[','').replace(']','').replace("'", "") )

train_data_df['actor_name'] = train_data_df.actor_name.apply(lambda actor: actor.split(',') )
train_data_df['director_name'] = train_data_df.director_name.apply(lambda director: director.split(',') )
train_data_df['writer_name'] = train_data_df.writer_name.apply(lambda writer: writer.split(',') )
train_data_df['country'] = train_data_df.country.apply(lambda country: country.split(',') )
train_data_df['genre'] = train_data_df.genre.apply(lambda genre: genre.split(',') )

In [7]:
train_data_df['actor_name'] = train_data_df.actor_name.apply(lambda actor: [a.strip() for a in actor])
train_data_df['director_name'] = train_data_df.director_name.apply(lambda director: [d.strip() for d in director])
train_data_df['writer_name'] = train_data_df.writer_name.apply(lambda writer: [w.strip() for w in writer] )
train_data_df['country'] = train_data_df.country.apply(lambda country: [c.strip() for c in country] )
train_data_df['genre'] = train_data_df.genre.apply(lambda genre: [g.strip() for g in genre])


train_data_df[train_data_df['movie_id'] == 'agony_(film)'].values

array([['agony_(film)', 'agony',
        list(['anatoli_romashin', 'alisa_freindlich', 'velta_līne', 'alexei_petrenko']),
        list(['elem_klimov']), list(['semyon_lungin', 'ilya_nusinov']),
        list(['russian_soviet_federative_socialist_republic', 'soviet_union']),
        list(['political_drama', 'biographical_film', 'historical_film_q17013749']),
        "['velta_līne', 'elem_klimov', 'alexei_petrenko', 'anatoli_romashin', 'biographical_film', 'soviet_union', 'historical_film_q17013749', 'ilya_nusinov', 'semyon_lungin', 'political_drama', 'russian_soviet_federative_socialist_republic', 'alisa_freindlich']"]],
      dtype=object)

In [8]:
train_data_df

,movie_id,title,actor_name,director_name,writer_name,country,genre,movie_sentence
0,'68_(film),'68,"[jan_němec, neil_young]",[steven_kovacs],[steven_kovacs],[united_states],[lgbt-related_film_q20442589],"['lgbt-related_film_q20442589', 'jan_němec', '..."
1,'71_(film),'71,"[richard_dormer, charlie_murphy_, sean_harris,...",[yann_demange],[gregory_burke],[united_kingdom],"[crime_film, thriller_film, war_film, action_f...","['gregory_burke', 'action_film', 'crime_film',..."
2,'gator_bait,'gator bait,[claudia_jennings],[beverly_sebastion],[beverly_sebastion],[united_states],"[thriller_film, action_film]","['action_film', 'claudia_jennings', 'thriller_..."
3,'neath_the_arizona_skies,'neath the arizona skies,"[john_wayne, sheila_terry]",[harry_l._fraser],[burl_r._tuttle],[united_states],[b_movies_],"['b_movies_', 'john_wayne', 'burl_r._tuttle', ..."
4,'r_xmas,'r xmas,"[andrew_fiscella, victor_argo, ice-t, drea_de_...",[abel_ferrara],[abel_ferrara],[united_states],"[christmas_film_q28026639, crime_film]","['crime_film', 'lillo_brancato', 'christmas_fi..."
...,...,...,...,...,...,...,...,...
19487,zouzou_(film),zouzou,"[jean_gabin, josephine_baker]",[marc_allégret],[carlo_rim],[france],[musical_film],"['marc_allégret', 'josephine_baker', 'jean_gab..."
19488,zubeidaa,zubeidaa,"[rekha, karisma_kapoor, farida_jalal, amrish_p...",[shyam_benegal],[khalid_mohamed],[india],[romance_film],"['farida_jalal', 'rekha', 'romance_film', 'amr..."
19489,zulu_(1964_film),zulu,"[ulla_jacobsson, stanley_baker, james_booth, m...",[cy_endfield],"[john_prebble, cy_endfield]",[united_kingdom],"[adventure_film, war_film]","['john_prebble', 'jack_hawkins', 'michael_cain..."
19490,zulu_(2013_film),zulu,"[forest_whitaker, orlando_bloom]",[jérôme_salle],[jérôme_salle],[france],[crime_film],"['crime_film', 'forest_whitaker', 'jérôme_sall..."


In [9]:
#create dictionary with the movie_id and title
movies_dict= dict(zip(train_data_df.movie_id, train_data_df.title))


# Manual Validation

In [12]:
#Manual check
#movie_id = 'godzilla_vs._biollante'
#movie_id = 'the_bunker_(2001_film)'
#movie_id= 'just_go_with_it'
#movie_id='agony_(film)'
#movie_id= 'harry_potter_and_the_half-blood_prince_(film)'
#movie_id = 'toy_story_4'
#movie_id = 'tsar_(film)'
#movie_id='high_school_musical_3:_senior_year'
#movie_id= 'hook_(film)'
#movie_id="destiny_(1997_film)"
#movie_id="blade:_trinity"


similar_movies = model.docvecs.most_similar(movie_id, topn=10)
sim_movies_list = [sim[0] for sim in similar_movies]

sim_df= pd.DataFrame(columns=['Title','Similarity Ratio'])

for sim in similar_movies:
    #title = helper_fn.find_movie_title_by_id(sim[0], train_data_df)
    data = {'Title':  [sim[0]],'Similarity Ratio': [sim[1]]}
    tmp_df = pd.DataFrame(data, columns=['Title','Similarity Ratio'])
    sim_df = pd.concat([sim_df, tmp_df])   

common_features_df = eval_fn.get_common_attrs_cnt_for_sim_list(train_data_df, sim_movies_list, movie_id) 

result_df = sim_df.merge(common_features_df, left_on= ['Title'], right_on=['Title'], how='inner')

result_df.Title= result_df.Title.map(lambda t: (helper_fn.find_movie_title_by_id(t, train_data_df).values[0]).capitalize())

query_movie_title = helper_fn.find_movie_title_by_id(movie_id, train_data_df).values[0].capitalize()
print("\n")
print("Similarity for the movie title \"{}\"".format(query_movie_title))
display(tabulate(result_df, headers=result_df.columns,tablefmt='html', showindex=False))

#sim = docs_vectors.most_similar(positive= 'harry_potter_and_the_chamber_of_secrets_(film)', topn= 5)
#eval_fn.print_sim_list(similar_movies)


TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
eval_fn.find_common_attrs(train_data_df, movie_id, 'blade_(film)')

In [ ]:
#Find the count of every feature
sim_movies_deails_df = train_data_df.loc[train_data_df.apply(lambda movie: True if movie.movie_id in sim_movies_list else False, axis=1)]

sim_movies_deails_df.apply(lambda movie: len(movie.actor_name), axis=1)

movie_record = train_data_df[train_data_df.movie_id == movie_id]

def find_feature_cnt_for_movie(movie_record, col):
    col_list = movie_record[col].to_list()
    if col_list is not None:
        return len(col_list[0])
    else:
        return 0

actors = find_feature_cnt_for_movie(movie_record, 'actor_name')

actors

In [ ]:
for s in similar_movies:
    print(s[1])
    

# Visualization

In [ ]:
#prepare data from the same movies used in the manual check
test_movies_list= ['godzilla_vs._biollante'
        ,'the_bunker_(2001_film)'
        ,'just_go_with_it'
        ,'agony_(film)'
        ,'harry_potter_and_the_half-blood_prince_(film)'
        ,'toy_story_4'
        ,'tsar_(film)'
        ,'high_school_musical_3:_senior_year'
        , 'hook_(film)'
        ,"destiny_(1997_film)"
        ]

In [ ]:
#Actors

In [ ]:
#query each movie for most similar list
#save each query movie and the result in a dictionary
#for each group, retrieve actors for all the movies in the list

group_by_q_movie=[]
all_groups_actors=[]
for movie_id in test_movies_list:
    sims= model.docvecs.most_similar(movie_id,topn=5)
    g_movies=[]
    for s in sims:
        g_movies.append(s[0])
    group = {'g_id':movie_id, 'g_movies':g_movies}
    group_by_q_movie.append(group)
    
    
for group in group_by_q_movie:
    query_movie_id = group['g_id']
    sim_movies_id_list = group['g_movies']
    group_actors=[]
    group_movies=[]
    actors = helper_fn.find_actors_by_movie_id(query_movie_id, train_data_df)
    
    group_actors = group_actors + actors[0]
    #print(group_actors)
    group_movies.append(query_movie_id)    
   # print("group ID {}".format(query_movie_id))
    
    for movie_id in sim_movies_id_list:
        #print("sim movie id {} :".format(movie_id))
        actors = helper_fn.find_actors_by_movie_id(movie_id, train_data_df)
        group_actors = group_actors + actors[0]
        group_movies.append(movie_id)
        #print(group_actors)
    all_groups_actors.append({'g_id':query_movie_id, 'g_movies':group_movies, 'g_actors':group_actors})


group_actors_df = pd.DataFrame(data= {'movie_id':[g['g_id'] for g in all_groups_actors], 
                'movies_list':[g['g_movies'] for g in all_groups_actors],
               'actors_list':[g['g_actors'] for g in all_groups_actors]
               })


#display(group_actors_df)

data_target_list=[]
movies_target_list=[]
actors_target_list=[]

for group in all_groups_actors:
    movies_target_list = movies_target_list + list(itertools.product([group['g_id']], group['g_movies']))
print(len(movies_target_list))
x_movies_df= pd.DataFrame(data= list(set(movies_target_list)), columns=['Y','X'])
x_movies_df['type']= 'movie'

    
for group in all_groups_actors:
    actors_target_list = actors_target_list + list(itertools.product([group['g_id']], group['g_actors']))
print(len(actors_target_list))
x_actors_df= pd.DataFrame(data= list(set(actors_target_list)), columns=['Y','X'])
x_actors_df['type']= 'actor'

final_df = x_actors_df.append(x_movies_df)

In [ ]:
###Visualization
print(len(final_df))

final_df.reset_index(drop=True, inplace= True)
y=[]
data_X = []
type_x =[]
final_df['title']= final_df.apply(lambda row: helper_fn.find_movie_title_by_id(row.Y,train_data_df).values[0], axis=1)
final_df['title'] = final_df['title'].str.capitalize()
#y_vec = [model[tag] for tag in y]

doctags= model.docvecs.doctags
vocab = model.wv.vocab
final_df = final_df.sample(frac=1)
plt.rcParams["figure.figsize"] = [8,6]
data_X_vec=[]
for tag in final_df['X'].values :                
    if tag in doctags:
        tag_vec= model[tag] 
    else:
        if tag in vocab:
            tag_vec= model.wv[tag] 
        else:
            tag_vec=[0]*50
    data_X_vec.append(tag_vec)
final_df['X_vec']=data_X_vec

tsne=  TSNE(perplexity= 50, n_components=2, init='pca', n_iter=7000, random_state=23)

tsne_obj= tsne.fit_transform(final_df.X_vec.to_list())

tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                        'Y':tsne_obj[:,1],
                        'movie':final_df.title.to_list(),
                         'X_type':final_df.type.to_list()})

#sns.set(rc={'figure.figsize':(11.7,8.27)})


#movies_only=tsne_df[tsne_df['X_type']=='actor']

In [ ]:
sns.set_context('paper')
palette=['purple','red','orange','brown','blue','dodgerblue','green','lightgreen','darkcyan', 'black']
g = sns.scatterplot(x="X", y="Y", data=tsne_df, palette= palette, hue="movie");

tsne_df.apply(lambda point: g.scatter(point['X'], point['Y'], marker=find_marker(point['X_type'])), axis=1)

              



g.legend(loc='center left', bbox_to_anchor=(1.00, 0.5), ncol=1)


In [ ]:
def find_marker(type_X):
    if type_X == 'movie':
        return "o"
    else:
        if type_X == 'actor':
            return "s"

In [ ]:
movie_id='agony_(film)'
sims= model.docvecs.most_similar(movie_id,topn=5)
eval_fn.calculate_BLEU_score(movie_id, sims,train_data_df)

In [ ]:
#visialize movies grouped by the query movie_id- for each movie in the test set, generate the sim list and save them in a dict with the query movie as the key
#for each group, plot the query movie and the sim list using distinct color
#generate similarity list for each movie in the test set
vis_group_by_q_movie=[]
for movie_id in test_movies_list:
    sims= model.docvecs.most_similar(movie_id)
    g_movies=[]
    for s in sims:
        g_movies.append(s[0])
    group = {'g_id':movie_id, 'g_movies':g_movies}
    vis_group_by_q_movie.append(group)
#print(vis_group_by_q_movie)    

data_target_list=[]
for group in vis_group_by_q_movie:
    data_target_list = data_target_list + list(itertools.product([group['g_id']], group['g_movies']))
y=[]
data_X = []

#print(data_target_list)

for pair in data_target_list:
    title = helper_fn.find_movie_title_by_id(pair[0],train_data_df)
    y.append(title.values[0].capitalize())
    data_X.append(pair[1])

data_X_vec = [model[tag] for tag in data_X]
tsne=  TSNE(perplexity= 5, n_components=2, init='pca', n_iter=5000, random_state=23)
tsne_obj= tsne.fit_transform(data_X_vec)
tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                        'Y':tsne_obj[:,1],
                        'movie':y})

sns.set_context("notebook")
g=sns.scatterplot(x="X", y="Y",
              hue="movie",
              palette=['purple','red','orange','brown','blue',
                       'dodgerblue','green','lightgreen','darkcyan', 'black'],
              legend='full',
              data=tsne_df);
plt.title('Model generated movies with similarity')
g.figure.savefig("generated_simialr_movies.png")

g.legend(loc='center left', bbox_to_anchor=(1.00, 0.5), ncol=1)

In [ ]:
def split_list(col):
    

In [ ]:
#further analysis to the results in visualizing the 10 movies and their simialr ones
#get list of genres for each group

all_group_genres_list = []
for group in vis_group_by_q_movie: 
    group_genres_list=[]
    id_genres_list = []
    #genres for the query movie
    movie_genres = helper_fn.find_movie_genre_by_id(group['g_id'], train_data_df)
    id_genres_list = id_genres_list + movie_genres.to_list()[0]
    
    for movie_id in group['g_movies']:
        movie_genres = helper_fn.find_movie_genre_by_id(movie_id, train_data_df)
        group_genres_list = group_genres_list + movie_genres.tolist()[0]
     #   print(group_genres_list[0])
    g_genres={'g_id':group['g_id'], 'q_genres':id_genres_list,'g_genres': list(set(group_genres_list))}
    all_group_genres_list.append(g_genres)

group_genres_df = pd.DataFrame(data= {'movie_id':[g['g_id'] for g in all_group_genres_list]
                                      ,'q_genres':[g['q_genres'] for g in all_group_genres_list]
                                      ,'genres_list':[g['g_genres'] for g in all_group_genres_list]})

group_genres_df['q_genres'] = group_genres_df.apply(lambda row: '|'.join(row['q_genres']), axis=1)
group_genres_df['genres_list'] = group_genres_df.apply(lambda row: '|'.join(row['genres_list']), axis=1)
display(tabulate(group_genres_df, headers=['Query Movie', "Query Movie Genres", "Similar movies genres"],tablefmt='html', showindex=False))
group_genres_df.to_excel("models/exp1/sim_gen_list.xlsx", index=False)
#group_genres_df

# Calculate the ranking score for a model

In [ ]:
#prepare test data
test_titles = []
test_titles= ['shrek','star trek','blade','toy story 4','pitch perfect','harry potter','godzilla','paddington','x-men']
test_data= eval_fn.build_movie_sequel_test_data(train_data_df, test_titles)

rank_score_list = eval_fn.calculate_rank_score(model,test_data,5)
group_rank_df = pd.DataFrame(data= {'group_id':[g['group_id'] for g in rank_score_list], 'score':[g['group_score'] for g in rank_score_list]})
group_rank_df.append(group_rank_df.sum(numeric_only=True), ignore_index=True)
group_rank_df.loc['Column_Total']= group_rank_df.sum(numeric_only=True, axis=0)
group_rank_df.iloc[len(group_rank_df)-1].group_id= "Total"

display(tabulate(group_rank_df, headers=['Group', "Rank Score"],tablefmt='html', colalign=("right", "left"),showindex=False))

In [ ]:
#Calculate model BLEU score
test_titles = []
test_titles= ['shrek','star trek','blade','toy story 4','pitch perfect','harry potter','godzilla','paddington','x-men']
test_data= eval_fn.build_movie_sequel_test_data(train_data_df, test_titles)
bleu_score = eval_fn.calculate_model_bleu_score(model, test_data, train_data_df, 5)
bleu_score

In [ ]:
len(vis_genres_sample_df)

## visualize movies by genre
#genres_sample_df contains movies from the genres 'Romantic_Comedy','Drama','Adventure','Crime','Action','Unknown'

In [ ]:
y=[]
data_X = []

#print(data_target_list)



vis_genres_sample_df = genres_sample_df.copy()

vis_genres_sample_df['data_X_vec']= vis_genres_sample_df.apply(lambda movie:model[movie.movie_id], axis=1)
vis_genres_sample_df['y']= vis_genres_sample_df.apply(lambda movie: assign_genre(genres_df, movie.movie_id), axis=1)

data_X_vec = list(vis_genres_sample_df['data_X_vec'].values)
y = vis_genres_sample_df['y']
tsne=  TSNE(perplexity= 5, n_components=2, init='pca', n_iter=5000, random_state=23)
tsne_obj= tsne.fit_transform(data_X_vec)
tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                        'Y':tsne_obj[:,1],
                        'movie':y})
tsne_df.head()
sns.scatterplot(x="X", y="Y",
              data=tsne_df);

g=sns.scatterplot(x="X", y="Y",
              hue="movie",
         #     palette=['purple','red','orange','brown','blue','black'],
              legend='full',
              data=tsne_df);
plt.title('Movies grouped by genre ')
g.legend(loc='center left', bbox_to_anchor=(1.00, 0.5), ncol=1)

## plot sequelmovies with some noise data with labels

In [ ]:
def tsne_plot_doc2vec(model, samples,movies_dict,height, width):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    

    for doc in samples:
        tokens.append(model[doc])
        labels.append(movies_dict[doc])
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(height, width)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
   # plt.ylim(1, -7)
   # plt.xlim(-7,-1)
    
    
    plt.title('Similar and non-similar movies')
    plt.show()
    #plt.figimage.savefig('movies_labels_large.png',  bbox_inches='tight')
    plt.close() 
    

In [ ]:
tsne_plot_doc2vec(model, sequel_sampled_data_df.movie_id,movies_dict,10, 10)

## Not sure what is this for

In [ ]:
def tsne_plot_doc2vec_categorized(model, samples,movies_dict, w,h):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    colors = []
    

    for doc in samples:
        tokens.append(model[doc])
        labels.append(movies_dict[doc])
        colors.append(assign_color(genres_df, doc))
    
    tsne_model = TSNE(perplexity=50, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(w, h)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i],color= colors[i][0] )
        plt.annotate('', #labels[i],
                     xy=(x[i], y[i], ),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
   # plt.xlim(-4.0, -1.0)
  #  display(HTML("<b>%d words most similar to '%s'</b>" % (n,word)))
    plt.title('Movies Distribution by Genre')
    plt.legend( labels=['Romantic_Comedy','Drama','Adventure','Crime','Action','Unknown'])
    plt.savefig('movies_genre.png')
   # plt.show(block=False)
    plt.show()
genres_sample_df

In [ ]:
tsne_plot_doc2vec_categorized(model, set(genres_sample_df[0:100].movie_id), movies_dict, 12,12)

In [ ]:
genre_samples= genres_sample_df.movie_id.values.tolist()
len(genre_samples)

In [ ]:
col_data=train_data_df[col_name]
val_list=[]
for data in col_data:
    for val in data:
        val_list.append(val)
occurence_count = Counter(val_list)
high_cnt = occurence_count.most_common(10)[0:10]
x= [cnt[0] for cnt in high_cnt ]


In [ ]:
tsne_plot_doc2vec(model, set(genre_samples[0:100]), movies_dict)

In [ ]:
from seaborn import sns

In [ ]:
#Plot movies by features

def plot_movies_by_feature(col_name, train_data_df, sample_size):
    col_data=train_data_df[col_name]
    val_list=[]
    for data in col_data:
        for val in data:
            val_list.append(val)

    occurence_count = Counter(val_list)
    print(len(occurence_count))
    high_cnt = occurence_count.most_common(sample_size)[0:sample_size]
    val_list= [cnt[0] for cnt in high_cnt ]
    print(val_list)
    
    movies_val_dict={}

    #val_list = random.sample(val_list,sample_size)
    for val in val_list:
        movies_val_dict[val] = helper_fn.find_movies_by_col_value(col_name, val, train_data_df)    
    
    data_target_list=[]
    for key in val_list:
        for val in movies_val_dict[key]:
            data_target_list.append((key,val))
    y=[]
    data_X = []

    for pair in data_target_list:
        y.append(pair[0])
        data_X.append(pair[1])

    data_X_vec = [model[tag] for tag in data_X]
    sns.set_context('notebook')
    tsne=  TSNE(perplexity= 50, n_components=2, init='pca', n_iter=5000, random_state=23)
    tsne_obj= tsne.fit_transform(data_X_vec)
    tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                        'Y':tsne_obj[:,1],
                        col_name:y})
    return tsne_df

def draw_plot(tsne_df, col_name, title,filename):
   # sns.set_palette(sns.color_palette("default", 20))
    #sns.color_palette()
    tsne_df[col_name] = tsne_df[col_name].apply(lambda c: c[1:] if c.startswith('_') else c)
    g=sns.scatterplot(x="X", y="Y",
                  hue=col_name,
              #    hue_order= cat_order,
                #  palette=['purple','red','orange'],
                  legend= 'full',            
                  data=tsne_df);
    
    g.set_title(title)
    #g.set_xlim(-600, 800)
    #g.set_ylim(-800, 600)
    g.legend(loc='center left', bbox_to_anchor=(1.00, 0.5), ncol=1)
    g.figure.savefig(filename)
    plt.close
    
        
    

In [ ]:
sample_size= 5
#col_name= 'actor_name' #10 actors
#col_name='writer_name' #10 samples
#col_name='director_name' #10
#col_name='country' #10 samples
#col_name='genre' #top 10

plot_df = plot_movies_by_feature(col_name,train_data_df, sample_size)
title= "Movies grouped by "+col_name
filename= col_name+"_plot.png"

In [ ]:
draw_plot(plot_df,col_name, title,filename )

In [ ]:
len(plot_df)

In [ ]:
def show_closest_line(similar_movies, query_movie, n):
   # display(html.HTML("<b>%d words most similar to '%s'</b>" % (n, query_movie)))
    
    tops = similar_movies
    
    items = [item[0] for item in tops]
    sims = [item[1] for i,item in enumerate(tops)]
    
    fig = plt.figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(111)

    
    plt.xticks(range(n), [i+1 for i in range(n)])
    plt.xlabel('Rank')
    plt.ylabel('Similarity')
    plt.xlim(-1,n)

    
    ax.plot(sims, color="purple", alpha=0.5)
    
    for item, x, y in zip(items, range(n), sims):
        ax.annotate( item, xy=(x, y), xytext=(20, -7), textcoords='offset points', 
                     ha='right', va='bottom', color='orange', fontsize=14 )

    
    
    
    plt.show()